# Hartree-Fock Analytic Nuclear Gradients Implementation

This notebook demonstrates how to compute analytic nuclear gradients for Hartree-Fock theory using Psi4Numpy.
The implementation follows the equations from Szabo and Ostlund's "Modern Quantum Chemistry".

## Theory Overview
For a closed-shell system, the Hartree-Fock energy gradient with respect to nuclear coordinate x is:

$$\frac{\partial E}{\partial x_i} = \frac{\partial E_{nuc}}{\partial x_i} + \sum_{\mu\nu} D_{\mu\nu} \frac{\partial h_{\mu\nu}}{\partial x_i} + \frac{1}{2} \sum_{\mu\nu\lambda\sigma} D_{\mu\nu} D_{\lambda\sigma} \frac{\partial (\mu\nu|\lambda\sigma)}{\partial x_i} - \sum_{\mu\nu} W_{\mu\nu} \frac{\partial S_{\mu\nu}}{\partial x_i}$$

equivalently,

$$ \frac{\partial E}{\partial x_i} = \frac{\partial E_{nuc}}{\partial x_i} + \sum_{\mu\nu} D_{\mu\nu} \frac{\partial h_{\mu\nu}}{\partial x_i} + \frac{1}{2} \sum_{\mu\nu\lambda\sigma} D_{\mu\nu} D_{\lambda\sigma} \frac{\partial (\mu\nu|\lambda\sigma)}{\partial x_i}- \sum_{pq} F_{pq} \frac{\partial S_{pq}}{\partial R_A} $$
and similarly for the $y$ and $z$ coordinates.  Importantly, each atom has $x$, $y$, and $z$ coordinates so this yields a gradient
vector with $3 N_{\rm atoms}$ elements.

Where:
- $E_{nuc}$ is the nuclear repulsion energy
- $D_{\mu\nu}$ is the density matrix 
- $h_{\mu\nu}$ is the one-electron integral matrix in the AO basis
- $(\mu\nu|\lambda\sigma)$ are the two-electron integrals in the AO basis
- $S_{\mu\nu}$ is the overlap matrix in the AO basis
- $W_{\mu\nu}$ is the energy-weighted density matrix
- $F_{p q}$ is the Fock matrix in the MO basis
- $S_{pq}$ is the overlap matrix in the MO basis

# Numerical Hartree-Fock gradients
The Hartree-Fock energy gradient may also be approximated using centered finite differences as follows:
$$\frac{\partial E}{\partial x_i} \approx \frac{ E(x_1, ..., x_i + \delta x, x_{i+1}, ... x_N) - E(x_1, ..., x_i - \delta x, x_{i+1}, ... x_N) }{2 \delta x} $$
where $\delta x$ is a sufficiently small step along the nuclear coordinates.  

**Importantly** Numerical gradients in this approximation require two full solutions to the RHF energy for each nuclear degree of freedom, so in total $6 N_{atoms}$ RHF calculations are needed for each numerical gradient evaluation.  The analytical gradients require only a single solution of the RHF energy and then a series of contractions using the existing RHF density matrix.  

Nevertheless, we can set up a cell to compute the numerical gradient as a reference.

In [2]:
# First import the necessary libraries
import psi4
import numpy as np
np.set_printoptions(precision=15, linewidth=200, suppress=True)

In [28]:
def modify_geometry_string(geometry_string, displacement_array):
    """
    Extracts Cartesian coordinates from a Psi4 geometry string, applies a
    transformation function to the coordinates, and returns a new geometry string.

    Args:
        geometry_string (str): A Psi4 molecular geometry string.
        transformation_function (callable): A function that takes a NumPy
            array of Cartesian coordinates (N x 3) as input and returns a
            NumPy array of the same shape with the transformed coordinates.

    Returns:
        str: A new Psi4 molecular geometry string with the transformed coordinates.
    """
    lines = geometry_string.strip().split('\n')
    atom_data = []
    symmetry = None

    for line in lines:
        line = line.strip()
        if not line:
            continue
        if line.lower().startswith("symmetry"):
            symmetry = line
            continue
        parts = line.split()
        if len(parts) == 4:
            atom = parts[0]
            try:
                x, y, z = map(float, parts[1:])
                atom_data.append([atom, x, y, z])
            except ValueError:
                # Handle cases where the line might not be atom coordinates
                pass

    if not atom_data:
        return ""

    coordinates = np.array([[data[1], data[2], data[3]] for data in atom_data])

    # Apply the transformation function
    transformed_coordinates = displacement_array  + coordinates

    new_geometry_lines = []
    for i, data in enumerate(atom_data):
        atom = data[0]
        new_geometry_lines.append(f"{atom} {transformed_coordinates[i, 0]:.8f} {transformed_coordinates[i, 1]:.8f} {transformed_coordinates[i, 2]:.8f}")

    new_geometry_string = "\n".join(new_geometry_lines)
    if symmetry:
        new_geometry_string += f"\n{symmetry}"

    return f"""{new_geometry_string}"""


def run_psi4_calculation(geometry_string, displacement_array, basis_set='sto-3g', method='scf'):
    """
    Runs a Psi4 calculation with the given geometry string, basis set, and method.

    Args:
        geometry_string (str): A Psi4 molecular geometry string.
        displacement_array (np.ndarray): An array of displacements to apply to the coordinates.
        basis_set (str): The basis set to use for the calculation, defaults to 'sto-3g'.
        method (str): The quantum chemistry method to use for the calculation, defaults to 'scf'.
        

    Returns:
        dict: A dictionary containing the results of the Psi4 calculation.
    """
    # Modify the geometry string with the displacement array
    modified_geometry_string = modify_geometry_string(geometry_string, displacement_array)

    # Set up Psi4 options
    psi4.set_options({
        'basis': basis_set,
        'scf_type': 'pk',
        'e_convergence': 1e-8,
        'd_convergence': 1e-8,
    })

    # Run the Psi4 calculation
    try:
        psi4.geometry(modified_geometry_string)
        energy = psi4.energy(method)
    except Exception as e:
        print(f"Error during Psi4 calculation: {e}")
        return None

    return energy

def compute_psi4_analytical_gradient(geometry_string, basis_set='sto-3g', method='scf'):
    """
    Computes the energy gradient for a given geometry string using Psi4.

    Args:
        geometry_string (str): A Psi4 molecular geometry string.
        basis_set (str): The basis set to use for the calculation, defaults to 'sto-3g'.
        method (str): The quantum chemistry method to use for the calculation, defaults to 'scf'.

    Returns:
        np.ndarray: The energy gradient as a NumPy array.
    """
    # Set up Psi4 options
    psi4.set_options({
        'basis': basis_set,
        'scf_type': 'pk',
        'e_convergence': 1e-8,
        'd_convergence': 1e-8,
    })

    # Run the Psi4 calculation
    try:
        psi4.geometry(geometry_string)
        gradient = psi4.gradient(method)
    except Exception as e:
        print(f"Error during Psi4 calculation: {e}")
        return None

    return np.asarray(gradient)

def compute_psi4_numerical_gradient(geometry_string, displacement_unit=0.01, basis_set='sto-3g', method='scf'):
    """
    NEEDS COMPLETING: Computes the numerical gradient for a given geometry string using Psi4.

    Args:
        geometry_string (str): A Psi4 molecular geometry string.
        displacement_unit (float): The unit of displacement for numerical gradient calculation, defaults to 0.01.
        basis_set (str): The basis set to use for the calculation, defaults to 'sto-3g'.
        method (str): The quantum chemistry method to use for the calculation, defaults to 'scf'.

    Returns:
        np.ndarray: The numerical gradient as a NumPy array.
    """
    # Set up Psi4 options
    psi4.set_options({
        'basis': basis_set,
        'scf_type': 'pk',
        'e_convergence': 1e-8,
        'd_convergence': 1e-8,
    })

    # Get the number of atoms from the geometry string
    num_atoms = len(geometry_string.strip().split('\n')) - 1

    # Initialize the numerical gradient array
    numerical_gradient = np.zeros((num_atoms, 3))

    # loop over all atoms
    for i in range(num_atoms):

        # loop over all three dimensions
        for j in range(3):
            # Create a displacement unit vector
            displacement_array = np.zeros((num_atoms, 3))
            displacement_array[i, j] = displacement_unit
            
            # print the displacement unit vector
            print(displacement_array)

            # Insert code to compute psi4 energy at forward displacement

            # Insert code to compute psi4 energy at backwards displacement

            # Insert code to compute finite difference along this displacement

            # Insert code to store this to the appropriate gradient element


    # return the gradient
    return None 


 


In [29]:
starting_string = """
O 0.0 0.0 0.0
H 0.0 0.757 0.587
H 0.0 -0.757 0.587
symmetry c1
"""

starting_displacement = np.array([[0, 0.0, 0.01], [0, 0, 0,], [0, 0, .0]])

# Run the Psi4 calculation
energy = run_psi4_calculation(starting_string, starting_displacement, basis_set='sto-3g', method='scf')
gradient = compute_psi4_analytical_gradient(starting_string, basis_set='sto-3g', method='scf')

# test the numerical gradient
numerical_gradient = compute_psi4_numerical_gradient(starting_string, displacement_unit=0.01, basis_set='sto-3g', method='scf')

[[0.01 0.   0.  ]
 [0.   0.   0.  ]
 [0.   0.   0.  ]]
[[0.   0.01 0.  ]
 [0.   0.   0.  ]
 [0.   0.   0.  ]]
[[0.   0.   0.01]
 [0.   0.   0.  ]
 [0.   0.   0.  ]]
[[0.   0.   0.  ]
 [0.01 0.   0.  ]
 [0.   0.   0.  ]]
[[0.   0.   0.  ]
 [0.   0.01 0.  ]
 [0.   0.   0.  ]]
[[0.   0.   0.  ]
 [0.   0.   0.01]
 [0.   0.   0.  ]]
[[0.   0.   0.  ]
 [0.   0.   0.  ]
 [0.01 0.   0.  ]]
[[0.   0.   0.  ]
 [0.   0.   0.  ]
 [0.   0.01 0.  ]]
[[0.   0.   0.  ]
 [0.   0.   0.  ]
 [0.   0.   0.01]]


In [21]:
print("Energy:", energy)
print("Gradient:\n", gradient)

Energy: -74.9617917984088
Gradient:
 [[ 0.                 0.000000000000001  0.061008877848977]
 [-0.                -0.023592244369028 -0.030504438924483]
 [ 0.                 0.023592244369025 -0.030504438924483]]


# Function to compute nuclear gradient
$$\frac{\partial E_{nuc}}{\partial x_i}$$ 


In [ ]:
def compute_numerical_gradient(geometry_string):
    """

    """
    # Define your molecular geometry
    molecule = psi4.geometry("""
    O 0.0 0.0 0.0
    H 0.0 0.757 0.587
    H 0.0 -0.757 0.587
    symmetry c1
    """)

    # get the nuclear repulsion gradients
    nuclear_repulsion_gradients = np.asarray(molecule.nuclear_repulsion_energy_deriv1())
    
    return None

# Function to compute Fock matrix and transform to the MO basis 
$$ F_{ij} = V_{ij} + T_{ij} + 2 \sum_{k}^{occ} (ii|kk) - \sum_{k}^{occ} (ik|kj) $$


In [ ]:
def compute_fock_matrix(geometry_string, basis_set='sto-3g', method='scf'):
    """

    """
    return None

In [1]:
import psi4
import numpy as np
np.set_printoptions(precision=15, linewidth=200, suppress=True)

# Define your molecule
molecule = psi4.geometry("""
O 0.0 0.0 0.0
H 0.0 0.757 0.587
H 0.0 -0.757 0.587
symmetry c1
""")

# set options
options = {'BASIS':'STO-3G', 'SCF_TYPE':'PK',
           'E_CONVERGENCE':1e-10,
           'D_CONVERGENCE':1e-10}
psi4.set_options(options)

# Run an RHF calculation
rhf_e, wfn = psi4.energy('SCF', molecule=molecule, return_wfn=True)

# Get basic information about the molecule 
n_atoms = molecule.natom()
occ = wfn.doccpi()[0]
nmo = wfn.nmo()

# get the transformation vectors as psi4 Matrix object
C = wfn.Ca_subset("AO", "ALL")

# store transformation vectors in numpy array
npC = np.asarray(C)

# Create a MintsHelper object
mints = psi4.core.MintsHelper(wfn.basisset())

# Get the core Hamiltonian in the ao basis
H_ao = np.asarray(mints.ao_kinetic()) + np.asarray(mints.ao_potential())

# Transform H to the MO basis
H = np.einsum('uj, vi, uv', npC, npC, H_ao)

# Integral generation from Psi4's MintsHelper
#MO = np.asarray(mints.mo_eri(C, C, C, C))
# Physicist notation    
#MO = MO.swapaxes(1,2)

#F = H + 2.0 * np.einsum('pmqm->pq', MO[:, :occ, :, :occ])
#F -= np.einsum('pmmq->pq', MO[:, :occ, :occ, :])

# Get ERI in the MO basis
ERI = np.asarray(mints.mo_eri(C, C, C, C))

# store in phycisist notation
ERI = ERI.swapaxes(1,2)

# Get the Fock matrix
F = H + 2 * np.einsum('pmqm->pq', ERI[:, :occ, :, :occ]) - np.einsum('pmmq->pq', ERI[:, :occ, :occ, :])

# Get the overlap derivative in MO basis
overlap_derivs = np.zeros((3 * n_atoms, nmo, nmo))
overlap_gradient = np.zeros(3 * n_atoms)

for atom_index in range(n_atoms):
    # Derivatives with respect to x, y, and z of the current atom
    for cart_index in range(3):
        deriv_index = 3 * atom_index + cart_index

        # Get overlap derivatives for this atom and Cartesian component
        overlap_derivs[deriv_index, :, :] = np.asarray(mints.mo_oei_deriv1("OVERLAP", atom_index, C, C)[cart_index])
        overlap_gradient[deriv_index] = -2.0 * np.einsum('ii,ii->', F[:occ, :occ], overlap_derivs[deriv_index, :occ, :occ])


print("Overlap Gradient")
print(overlap_gradient.reshape(3,3))


Scratch directory: /tmp/
   => Libint2 <=

    Primary   basis highest AM E, G, H:  6, 6, 3
    Auxiliary basis highest AM E, G, H:  7, 7, 4
    Onebody   basis highest AM E, G, H:  -, -, -
    Solid Harmonics ordering:            Gaussian

*** tstart() called on CHEM9QDFT72ALT
*** at Thu Apr 24 09:00:09 2025

   => Loading Basis Set <=

    Name: STO-3G
    Role: ORBITAL
    Keyword: BASIS
    atoms 1   entry O          line    81 file /Users/jfoley19/miniconda3/envs/p4env/share/psi4/basis/sto-3g.gbs 
    atoms 2-3 entry H          line    19 file /Users/jfoley19/miniconda3/envs/p4env/share/psi4/basis/sto-3g.gbs 


         ---------------------------------------------------------
                                   SCF
               by Justin Turney, Rob Parrish, Andy Simmonett
                          and Daniel G. A. Smith
                              RHF Reference
                        1 Threads,    500 MiB Core
         -------------------------------------------------------

In [ ]:
nuclear_repulsion_gradients = np.asarray(molecule.nuclear_repulsion_energy_deriv1())

overlap_derivs = np.zeros((3 * n_atoms, nmo, nmo))
kinetic_derivs = np.zeros((3 * n_atoms, nmo, nmo))
potential_derivs = np.zeros((3 * n_atoms, nmo, nmo))
eri_derivs = np.zeros((3 * n_atoms, nmo, nmo))

# contract the eri derivs to J_deriv and K_deriv
J_deriv = np.zeros((3 * n_atoms, nmo, nmo))
K_deriv = np.zeros((3 * n_atoms, nmo, nmo))

for atom_index in range(n_atoms):
    # Derivatives with respect to x, y, and z of the current atom
    for cart_index in range(3):
        deriv_index = 3 * atom_index + cart_index

        # Get overlap derivatives for this atom and Cartesian component
        overlap_derivs[deriv_index, :, :] = np.asarray(mints.ao_oei_deriv1("OVERLAP", atom_index)[cart_index])

        # Get kinetic energy derivatives
        kinetic_derivs[deriv_index, :, :] = np.asarray(mints.ao_oei_deriv1("KINETIC", atom_index)[cart_index])

        # Get potential energy derivatives
        potential_derivs[deriv_index, :, :] = np.asarray(mints.ao_oei_deriv1("POTENTIAL", atom_index)[cart_index])

        # get two electron integral derivatives
        eri_derivs[deriv_index, :, :, :, :] = np.asarray(mints.ao_tei_deriv1(atom_index)[cart_index]) 

print("Overlap Integral Derivatives (Shape):", overlap_derivs.shape)
print("Kinetic Energy Integral Derivatives (Shape):", kinetic_derivs.shape)
print("Nuclear Attraction Integral Derivatives (Shape):", potential_derivs.shape)
print("Electron Repulsion Integral (Shape):", eri_derivs.shape)

print("Density matrix")
print(D * 2)

In [ ]:
# contract ERI_deriv to J_deriv and K_deriv
for atom_index in range(n_atoms):
    for cart_index in range(3):
        deriv_index = 3 * atom_index + cart_index
        tei_deriv = eri_derivs[deriv_index]
        J_deriv[deriv_index, :, :] += 2 * np.einsum("pqrs,rs->pq", tei_deriv, D)
        K_deriv[deriv_index, :, :] -= np.einsum("prqs,rs->pq", tei_deriv, D)
        

In [ ]:
# Build the Fock matrix at the converged geometry
fock_matrix = np.asarray(rhf_wfn.Fa())

# Build the energy-weighted density matrix
energy_weighted_density = W


electronic_gradient = np.zeros(3 * n_atoms)
kinetic_gradient = np.zeros(3 * n_atoms)
potential_gradient = np.zeros(3 * n_atoms)
overlap_gradient = np.zeros(3 * n_atoms)
coulomb_gradient = np.zeros(3 * n_atoms)
exchange_gradient = np.zeros(3 * n_atoms)


for atom_index in range(n_atoms):
    for cart_index in range(3):
        deriv_index = 3 * atom_index + cart_index

        # Contribution from one-electron integral derivatives
        h_deriv = kinetic_derivs[deriv_index] + potential_derivs[deriv_index]
        electronic_gradient[deriv_index] += np.sum(density_matrix * h_deriv)
        kinetic_gradient[deriv_index] += np.sum(density_matrix * kinetic_derivs[deriv_index])
        potential_gradient[deriv_index] += np.sum(density_matrix * potential_derivs[deriv_index])

        coulomb_gradient[deriv_index] += np.sum(D * J_deriv[deriv_index])
        exchange_gradient[deriv_index] += np.sum(D * K_deriv[deriv_index])

        # Contribution from two-electron integral derivatives
        #J = np.einsum("pqrs,rs->pq", I, D)
        #K = np.einsum("prqs,rs->pq", I, D)
        #tei_deriv = eri_derivs[deriv_index]
        #print("Printing TEI_DERIV Shape")
        #print(tei_deriv.shape)
        #print("Printing D shape")
        #print(D.shape)
        #coulomb_gradient[deriv_index] += 2 * np.einsum("pqrs,rs->pq", tei_deriv, D)
        #exchange_gradient[deriv_index] -= np.einsum("prqs,rs->pq", tei_deriv, D)
        #electronic_gradient[deriv_index] += 0.5 * np.sum(density_matrix[:, :, None, None] * density_matrix[None, None, :, :] * tei_deriv)
        #coulomb_plus_exchange_gradient[deriv_index] += 0.5 * np.sum(density_matrix[:, :, None, None] * density_matrix[None, None, :, :] * tei_deriv)
        electronic_gradient[deriv_index] += coulomb_gradient[deriv_index] + exchange_gradient[deriv_index]
        # Pulay contribution from overlap derivative
        overlap_deriv = overlap_derivs[deriv_index]
        electronic_gradient[deriv_index] -= np.sum(energy_weighted_density * overlap_deriv)
        overlap_gradient[deriv_index] -= np.sum(density_matrix * overlap_deriv)

print("Electronic Energy Gradient:\n", electronic_gradient.reshape(3,3))
print("Nuclear Energy Gradient:\n", nuclear_repulsion_gradients)
print("Total Gradient:\n", electronic_gradient.reshape(3,3) + nuclear_repulsion_gradients)

## From output of this script https://github.com/psi4/psi4/blob/master/samples/psi4numpy/rhf-gradient/input.py
## when the current H2O geometry is used
_expected_nuclear_gradient = np.array([
    [0.00000000000000,  0.00000000000000,  2.99204046891092],
    [0.00000000000000, -2.05144597283373, -1.49602023445546],
    [0.00000000000000,  2.05144597283373, -1.49602023445546],
])

_overlap_gradient = np.array([
    [-0.00000000000000, -0.00000000000000,  0.30728746121587],
    [ 0.00000000000000, -0.14977126575800, -0.15364373060793],
    [-0.00000000000000,  0.14977126575800, -0.15364373060793],
])

_potential_gradient = np.array([
    [-0.00000000000000,  0.00000000000002, -6.81982772856799],
    [-0.00000000000000,  4.38321774316664,  3.40991386428399],
    [ 0.00000000000000, -4.38321774316666,  3.40991386428400],
])

_kinetic_gradient = np.array([
    [ 0.00000000000000, -0.00000000000000,  0.66968290617933],
    [ 0.00000000000000, -0.43735698924315, -0.33484145308966],
    [-0.00000000000000,  0.43735698924315, -0.33484145308967],
])

_coulomb_gradient = np.array([
    [ 0.00000000000000, -0.00000000000002,  3.34742251141627],
    [ 0.00000000000000, -2.03756324433539, -1.67371125570813],
    [-0.00000000000000,  2.03756324433541, -1.67371125570814],
])

_exchange_gradient = np.array([
    [-0.00000000000000,  0.00000000000000, -0.43559674130726],
    [-0.00000000000000,  0.26932748463493,  0.21779837065363],
    [ 0.00000000000000, -0.26932748463493,  0.21779837065363],
])

# Compare the calculated gradients with the expected values
print("Nuclear Gradients Agree:\n")
print(np.allclose(nuclear_repulsion_gradients.reshape(3,3), _expected_nuclear_gradient))

print("Overlap Gradients Agree:\n")
print(np.allclose(overlap_gradient.reshape(3,3), _overlap_gradient))
print("Printing Overlap Gradient")
print(overlap_gradient.reshape(3,3))
print("Printing Expected Overlap Gradient")
print(_overlap_gradient)

print("Kinetic Gradients Agree:\n")
print(np.allclose(kinetic_gradient.reshape(3,3), _kinetic_gradient))

print("Potential Gradients Agree:\n")
print(np.allclose(potential_gradient.reshape(3,3), _potential_gradient))

print("Coulomb Gradients Agree:\n")
print(np.allclose(coulomb_gradient.reshape(3,3), _coulomb_gradient))
print("Printing Coulomb Gradient")
print(coulomb_gradient.reshape(3,3))
print("Printing Expected Coulomb Gradient")
print(_coulomb_gradient)

print("Exchange Gradients Agree:\n")
print(np.allclose(exchange_gradient.reshape(3,3), _exchange_gradient))
print("Printing Exchange Gradient")
print(exchange_gradient.reshape(3,3))
print("Printing Expected Exchange Gradient")
print(_exchange_gradient)

#_jk_gradient = _coulomb_gradient + _exchange_gradient

#print(coulomb_gradient.reshape(3,3) - _coulomb_gradient)
#print(exchange_gradient.reshape(3,3) - _exchange_gradient)